## Notes:
- Doesnt appear to be training very well, training acc rises very slowly
- Needs to be tuned

Again will only run on Colab because of the loop moving memory onto the GPU

In [ ]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from torch.utils.data import DataLoader
from torch.autograd import Variable

### Data Loading

In [ ]:
!git clone -b @olly-branch "https://github.com/BSteiner1/ST311-Group-Project/"

Cloning into 'ST311-Group-Project'...
remote: Enumerating objects: 2199, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 2199 (delta 88), reused 158 (delta 49), pack-reused 1998
Receiving objects: 100% (2199/2199), 1.20 GiB | 38.73 MiB/s, done.
Resolving deltas: 100% (88/88), done.
Updating files: 100% (2021/2021), done.


In [ ]:
class_dict = {'blues' : 0,
              'classical': 1,
              'country': 2,
              'disco' : 3,
              'hiphop' : 4,
              'jazz' : 5,
              'metal' : 6,
              'pop' : 7,
              'reggae': 8,
              'rock' : 9    
}

In [ ]:
def collect_data():
    data = []
    # Collect all images from the 10 genres
    for genre in os.listdir("ST311-Group-Project/Data/images_original"):
        for image in os.listdir("ST311-Group-Project/Data/images_original/" + genre):
            image_path = "ST311-Group-Project/Data/images_original/" + genre + "/" + image
            grayscale_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            grayscale_img = grayscale_img[35:-36:, 54:-43]
            grayscale_img = torch.tensor(grayscale_img).to(dtype = torch.float32).clone().detach().requires_grad_(True)  #.requires_grad(True)
            #grayscale_img = torch.tensor(grayscale_img, requires_grad=True)
            # Splitting each image vertically into 5 different parts
            for i in range(5):
                # The data loader adds a dimension so I've removed a dimension from the image and class
                data.append((grayscale_img[ : ,  (67*i):67*(i+1)].reshape(1,217,67), torch.tensor(class_dict.get(genre)))) #.reshape(1)))
            
            #data.append((grayscale_img.reshape(1, 217, 335), torch.tensor(class_dict.get(genre))))
    return data     

data = collect_data()

In [ ]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### CNN

In [ ]:
'''
Net

'''


#Define the net
net3 = nn.Sequential(
    
    #16@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #32@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #64@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #128@3x3 Convolution Layer, Dropout 0.2
    nn.Conv2d(64,128, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(4x4) Pooling Layer with stride 4
    nn.MaxPool2d(kernel_size = 4, stride = 4),

    #64@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(128, 64, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #Flatten Layer
    nn.Flatten(),

    #768x384 FC Layer, Drop-out 0.2
    nn.Linear(768, 384), nn.Dropout(0.2), nn.ReLU(),

    #384x128 FC Layer
    nn.Linear(384, 128), nn.ReLU()

)

### RNN

In [ ]:
'''
Net
'''

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()

        #Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        #Hidden dimensions
        self.hidden_dim = hidden_dim

        #Number of hidden layers
        self.layer_dim = layer_dim

        #RNN Layer 
        self.rnn = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional = True)

        #Linear Layer
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x):

        #Initialize hidden state with zeros
        # to.('cuda') ensures the hidden and input tensors are GPU

        x = self.pool(x)

        h0 = torch.zeros(self.layer_dim*2, x.size(0), self.hidden_dim).requires_grad_().to('cuda')

        #RNN Foward Step
        out, hn = self.rnn(x, h0.detach())

        #Clips off final hidden state 
        out = self.fc(out[:, -1, :]) 

        return out

In [ ]:
input_dim = 108
hidden_dim = 169
layer_dim = 2
output_dim = 10

rnn_model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim).to('cuda')

### Ensemble (Acc. 56%)

Net

In [ ]:
class cnn_rnn(nn.Module):
    def __init__(self):
        super(cnn_rnn, self).__init__()

        #Net to process images
        self.net3 = net3

        #Net to process features
        self.rnn = rnn_model

        self.fc1 = nn.Linear(20, 10)

    def forward(self, x):

        #Pass image through cnn
        out1 = self.net3(x)

        #Reshape for RNN
        seq_dim = 67
        input_dim = 217 

        x_rnn = x.view(-1, seq_dim, input_dim).requires_grad_()

        #Pass features through rnn
        out2 = self.rnn(x_rnn)

        #Concatenate
        concat = torch.cat((out1, out2), dim = 1)
        
        #Pass through linear layer
        out = self.fc1(concat)

        return out

Training

In [ ]:
#Define instance of ensemble

cnn_rnn = cnn_rnn().to('cuda')

In [ ]:
#Define Loss and Optimiser

cnn_rnn_optimizer = torch.optim.Adam(cnn_rnn.parameters(), lr=0.001)
cnn_rnn_loss = nn.CrossEntropyLoss()

In [ ]:
#Define Epochs 

cnn_rnn_epochs = 20

In [ ]:
#Main Loop

for epoch in range(cnn_rnn_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
      
        #Moves inputs to the gpu
        inputs = inputs.to('cuda')
        targets = targets.to('cuda')

        # Zero the gradients
        cnn_rnn_optimizer.zero_grad()
        
        # Forward pass
        outputs = cnn_rnn(inputs)
        loss = cnn_rnn_loss(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        cnn_rnn_optimizer.step()
        
        # Compute running loss and accuracy
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    # Compute validation loss and accuracy
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:

            #Moves inputs to gpu
            inputs = inputs.to('cuda')
            targets = targets.to('cuda')
            
            outputs = cnn_rnn(inputs)
            loss = cnn_rnn_loss(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
    
    # Print epoch statistics
    print('Epoch %d: Train Loss: %.3f | Train Acc: %.3f%% | Val Loss: %.3f | Val Acc: %.3f%%'
          % (epoch+1, running_loss/len(train_loader), 100*correct/total,
             val_loss/len(test_loader), 100*val_correct/val_total))

### Sequential CNN-RNN (65%)


In [ ]:
'''
Net

'''

#Define the net
seq_net3 = nn.Sequential(
    
    #16@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(1, 16, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #32@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #64@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(2x2) Pooling Layer with stride 2
    nn.MaxPool2d(kernel_size=2, stride=2),

    #128@3x3 Convolution Layer, Dropout 0.2
    nn.Conv2d(64,128, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #(4x4) Pooling Layer with stride 4
    nn.MaxPool2d(kernel_size = 4, stride = 4),

    #64@3x3 Convolution Layer, Drop-out 0.2
    nn.Conv2d(128, 64, kernel_size = 3, stride = 1, padding = "same"), nn.Dropout(0.2), nn.ReLU(),

    #Flatten Layer
    nn.Flatten(),

    #768x384 FC Layer, Drop-out 0.2
    nn.Linear(768, 384), nn.Dropout(0.2), nn.ReLU(),

    #384x128 FC Layer
    nn.Linear(384, 128), nn.ReLU()

)

In [ ]:
'''
Net
'''

class seq_RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(seq_RNNModel, self).__init__()

        #Hidden dimensions
        self.hidden_dim = hidden_dim

        #Number of hidden layers
        self.layer_dim = layer_dim

        #Bi-GRU Layer 
        self.bigru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
  
        #hidden_dim x output_dim Linear Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        #Initialize hidden state with zeros
        # to.('cuda') ensures the hidden and input tensors are GPU

        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to('cuda')

        #RNN Foward Step
        out, hn = self.bigru(x, h0.detach())

        #Clips off final hidden state 
        out = self.fc(out[:, -1, :]) 

        #print(out.size())

        return out

In [ ]:
class seq_cnn_rnn(nn.Module):
    def __init__(self):
        super(seq_cnn_rnn, self).__init__()

        #Net to process images
        self.seq_net3 = seq_net3

        #Net to process features
        self.seq_rnn = seq_rnn_model

        #Final (10x10) linear layer
        self.fc1 = nn.Linear(10, 10)

    def forward(self, x):

        #Pass image through cnn
        cnn_out = self.seq_net3(x)

        #print(x.size())

        #Reshape for RNN
        seq_dim = 128
        input_dim = 1 

        cnn_out = cnn_out.view(-1, seq_dim, input_dim).requires_grad_()

        #Pass features through rnn
        rnn_out = self.seq_rnn(cnn_out)
        
        #Pass through linear layer
        out = self.fc1(rnn_out)

        return out

#Define RNN Structural Parameters
seq_input_dim = 1
seq_hidden_dim = 128
seq_layer_dim = 3
seq_output_dim = 10

#Define instance of model
seq_rnn_model = seq_RNNModel(seq_input_dim, seq_hidden_dim, seq_layer_dim, seq_output_dim).to('cuda')

#Transfer model to GPU
seq_cnn_rnn = seq_cnn_rnn().to('cuda')

#Define model functions
seq_cnn_rnn_optimizer = torch.optim.Adam(seq_cnn_rnn.parameters(), lr=0.0001)
seq_cnn_rnn_loss = nn.CrossEntropyLoss()

#Define epochs
seq_cnn_rnn_epochs = 100

In [ ]:
#Main Loop

for epoch in range(seq_cnn_rnn_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
      
        #Moves inputs to the gpu
        inputs = inputs.to('cuda')
        targets = targets.to('cuda')

        # Zero the gradients
        seq_cnn_rnn_optimizer.zero_grad()
        
        # Forward pass
        outputs = seq_cnn_rnn(inputs)
        loss = seq_cnn_rnn_loss(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        seq_cnn_rnn_optimizer.step()
        
        # Compute running loss and accuracy
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    # Compute validation loss and accuracy
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:

            #Moves inputs to gpu
            inputs = inputs.to('cuda')
            targets = targets.to('cuda')
            
            outputs = seq_cnn_rnn(inputs)
            loss = seq_cnn_rnn_loss(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
    
    # Print epoch statistics
    print('Epoch %d: Train Loss: %.3f | Train Acc: %.3f%% | Val Loss: %.3f | Val Acc: %.3f%%'
          % (epoch+1, running_loss/len(train_loader), 100*correct/total,
             val_loss/len(test_loader), 100*val_correct/val_total))

Epoch 1: Train Loss: 2.138 | Train Acc: 17.117% | Val Loss: 2.063 | Val Acc: 20.020%
Epoch 2: Train Loss: 1.956 | Train Acc: 25.300% | Val Loss: 1.932 | Val Acc: 26.426%
Epoch 3: Train Loss: 1.850 | Train Acc: 29.630% | Val Loss: 1.905 | Val Acc: 27.127%
Epoch 4: Train Loss: 1.790 | Train Acc: 31.757% | Val Loss: 1.862 | Val Acc: 29.129%
Epoch 5: Train Loss: 1.734 | Train Acc: 36.161% | Val Loss: 1.777 | Val Acc: 33.734%
Epoch 6: Train Loss: 1.681 | Train Acc: 38.413% | Val Loss: 1.742 | Val Acc: 34.434%
Epoch 7: Train Loss: 1.658 | Train Acc: 38.313% | Val Loss: 1.753 | Val Acc: 34.535%
Epoch 8: Train Loss: 1.605 | Train Acc: 41.742% | Val Loss: 1.681 | Val Acc: 37.437%
Epoch 9: Train Loss: 1.550 | Train Acc: 44.094% | Val Loss: 1.719 | Val Acc: 35.836%
Epoch 10: Train Loss: 1.514 | Train Acc: 44.845% | Val Loss: 1.599 | Val Acc: 41.141%
Epoch 11: Train Loss: 1.437 | Train Acc: 48.774% | Val Loss: 1.554 | Val Acc: 44.945%
Epoch 12: Train Loss: 1.381 | Train Acc: 53.003% | Val Loss: 1.

KeyboardInterrupt: ignored

RuntimeError: ignored